In [1]:
import cv2
import time
import os
import tensorflow as tf
import numpy as np

from tensorflow.python.keras.utils.data_utils import get_file

In [2]:
class Detector:
    def __init__(self):
        pass

    def saveModel(self, save_path):
            tf.saved_model.save(self.model, save_path)  # self.model'inizi kaydettiğiniz yer, save_path parametresiyle belirtilir
            print(f"Model saved at {save_path}")

    def convertToTFLite(self, saved_model_path, tflite_model_path):
        # Yüklenen modelin TFLite formatına dönüştürülmesi
        converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]  # İsteğe bağlı: Model optimizasyonları
        tflite_model = converter.convert()
        
        with open(tflite_model_path, 'wb') as f:
            f.write(tflite_model)
        print(f"TFLite model saved at {tflite_model_path}")

    def readClasses(self,classesFilePath):
        with open(classesFilePath,'r') as f:
            self.classesList = f.read().splitlines()

        #ColorList
        self.colorList = np.random.uniform(low=0,high=255,size=(len(self.classesList)))
        print(len(self.classesList),len(self.colorList))

    def downloadModel(self,modelURL):
        fileName = os.path.basename(modelURL)
        self.modelName = fileName[:fileName.index('.')]
        
        self.cacheDir = "./pretrained_models"
        os.makedirs(self.cacheDir,exist_ok=True)

        get_file(fname=fileName,origin=modelURL,cache_dir=self.cacheDir,cache_subdir="checkpoints",extract=True)


    def loadModel(self):
        print("Loading model "+self.modelName)
        tf.keras.backend.clear_session()
        self.model = tf.saved_model.load(os.path.join(self.cacheDir,"checkpoints",self.modelName,"saved_model"))
        print("Model "+self.modelName+" loaded succesfully! ")


    def createBoundingBox(self,image,threshold=0.5):
        inputTensor = cv2.cvtColor(image.copy(),cv2.COLOR_BGR2RGB)
        inputTensor = tf.convert_to_tensor(inputTensor,dtype=tf.uint8)
        inputTensor = inputTensor[tf.newaxis,...]

        detections = self.model(inputTensor)

        bboxs = detections["detection_boxes"][0].numpy()
        classIndexes = detections["detection_classes"][0].numpy().astype(np.int32)
        classScores = detections["detection_scores"][0].numpy()

        imH,imW,imC = image.shape
        bboxIdx = tf.image.non_max_suppression(bboxs,classScores,max_output_size=50,iou_threshold=threshold,score_threshold=threshold)

        print(bboxIdx)

        if len(bboxIdx) !=0:
            for i in bboxIdx:
                bbox = tuple(bboxs[i].tolist())
                classConfidence = round(100*classScores[i])
                classIndex = classIndexes[i]

                classLabelText = self.classesList[classIndex]
                classColor = self.colorList[classIndex]

                displayText = '{}:{}%'.format(classLabelText,classConfidence)
                ymin,xmin,ymax,xmax = bbox
            
                xmin,xmax,ymin,ymax = (xmin*imW,xmax*imW,ymin*imH,ymax*imH)
                xmin,xmax,ymin,ymax = int(xmin),int(xmax),int(ymin),int(ymax)

                cv2.rectangle(image,(xmin,ymin),(xmax,ymax),color=classColor,thickness=1)
                cv2.putText(image,displayText,(xmin,ymin-10),cv2.FONT_HERSHEY_PLAIN,1,classColor,2)
                #####################################################################
                lineWidth = min(int((xmax-xmin)*0.2),int((ymax-ymin)*0.2))

                cv2.line(image,(xmin,ymin),(xmin+lineWidth,ymin),classColor,thickness=5)
                cv2.line(image,(xmin,ymin),(xmin,ymin+lineWidth),classColor,thickness=5)
                
                cv2.line(image,(xmax,ymin),(xmax-lineWidth,ymin),classColor,thickness=5)
                cv2.line(image,(xmax,ymin),(xmax,ymin+lineWidth),classColor,thickness=5)

                #####################################################################   
                cv2.line(image,(xmin,ymax),(xmin+lineWidth,ymax),classColor,thickness=5)
                cv2.line(image,(xmin,ymax),(xmin,ymax-lineWidth),classColor,thickness=5)
                
                cv2.line(image,(xmax,ymax),(xmax-lineWidth,ymax),classColor,thickness=5)
                cv2.line(image,(xmax,ymax),(xmax,ymax-lineWidth),classColor,thickness=5)
                     

            return image

    def predictImage(self,imagePath,threshold=0.5):
        image = cv2.imread(imagePath)

        bboxImage = self.createBoundingBox(image,threshold)

        cv2.imwrite(self.modelName+".jpg",bboxImage)
        cv2.imshow("Result",bboxImage)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    def predictVideo(self,videoPath,threshold=0.5):
        cap = cv2.VideoCapture(videoPath)

        if(cap.isOpened()==False):
            print("Error opening file...")
            return
        
        (success,image) = cap.read()
        startTime=0

        while success:
            currentTime = time.time()
            fps = 1/(currentTime-startTime)
            startTime = currentTime

            bboxImage = self.createBoundingBox(image,threshold)
            cv2.putText(bboxImage,"FPS : "+str(int(fps)),(20,70),cv2.FONT_HERSHEY_PLAIN,2,(0,255,0),2)
            cv2.imshow("Result",bboxImage)

            key = cv2.waitKey(1) & 0xFF
            if key == ord("q"):
                break

            (success,image) = cap.read()
        cv2.destroyAllWindows()
            


        

In [3]:
classFile = "coco.names"        # BUNU DEĞİŞTİR !!!!!!!!!!!!!!!!
modelURL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz"
#modelURL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_1024x1024_coco17_tpu-8.tar.gz"
imagePath = "2.jpg"
videoPath = "1.mp4"
save_path = "./saved_model"  # Kaydedilecek dizin yolu
saved_model_path = "./saved_model"  # Kaydedilen modelin yolu
tflite_model_path = "./model.tflite"  # TFLite modelinin kaydedileceği yol



threshold = 0.5
saved_model_path = "./saved_model"  # Kaydedilen modelin yolu
tflite_model_path = "./model.tflite"  # TFLite modelinin kaydedileceği yol
detector = Detector()
detector.readClasses(classFile)
detector.downloadModel(modelURL)
detector.loadModel()
detector.predictImage(imagePath,threshold)
detector.saveModel(save_path)
detector.convertToTFLite(saved_model_path, tflite_model_path)

80 80
46047232/46042990 [==============================] - 7s 0us/step
Loading model ssd_mobilenet_v2_320x320_coco17_tpu-8
Model ssd_mobilenet_v2_320x320_coco17_tpu-8 loaded succesfully! 
tf.Tensor([0 1], shape=(2,), dtype=int32)
INFO:tensorflow:Assets written to: ./saved_model\assets


INFO:tensorflow:Assets written to: ./saved_model\assets


Model saved at ./saved_model
TFLite model saved at ./model.tflite


In [4]:
#detector.predictVideo(videoPath,threshold)